In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import cv2
def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (224,224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
# Useful functions for bilinear product
def outer_product(x):
    return K.batch_dot(x[0], x[1], axes = [1, 1]) / x[0].get_shape().as_list()[1]
def signed_sqrt(x):
    return K.sign(x)*K.sqrt(K.abs(x) + 1e-9)
def l2_normalize(x, axis = -1):
    return K.l2_normalize(x, axis=axis)

In [ ]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    model = Model(inputs=layers[0].input, outputs=x) ### inputs instead of input in this version
    return model

In [ ]:
from tensorflow.keras.initializers import glorot_normal
from keras.layers import Lambda
import tensorflow

# Model Architecture
def VGG16_outerproduct(filter_zero
                ,value_filter, classes, dropoutrate, model_name):
    K.clear_session()
    
    # VGG16 as base for extract features
#     base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = "imagenet")
    
    tensor_input = tensorflow.keras.layers.Input(shape=[224,224,3])
    base_model = tensorflow.keras.applications.vgg16.VGG16(
                            input_tensor=tensor_input
                            , include_top=False
                            , weights='imagenet'
                        )
    
    
    arryWeights_last_After =[]
    for i in range(512):
      arryWeights_last_After.append(i)
    for i in range(512):
      arryWeights_last_After[i]=1
    
    arryWeights_last_After[filter_zero]=value_filter
    # print('filter' , filter_zero, "=", arryWeights_last_After[filter_zero])
    def custom_layer_last(tensor):
        return tensor * arryWeights_last_After
    lambda_layer = Lambda (custom_layer_last, name="lambda_New")

    base_model = insert_intermediate_layer_in_keras (base_model, 18, lambda_layer)
    # base_model.summary()
    
    # Get output from VGG16 end layer features
    feature_model_1 = base_model
    x1 = feature_model_1.layers[18].output
    x1_shape = feature_model_1.layers[18].output_shape
    
    feature_model_2 = base_model
    x2 = feature_model_2.layers[18].output
    x2_shape = feature_model_2.layers[18].output_shape
    
    # reshape to (batch_size, total_pixels, filter_size) for outer product
    x1 = Reshape(
        [x1_shape[1]*x1_shape[2], x1_shape[-1]])(x1)
    x2 = Reshape(
        [x2_shape[1]*x2_shape[2], x2_shape[-1]])(x2)
    
    # Outer product of VGG16 features
    x = Lambda(outer_product)([x1, x2])
    x = Reshape([x1_shape[-1]*x2_shape[-1]])(x)
    x = Lambda(signed_sqrt)(x)
    x = Lambda(l2_normalize)(x)
    
    # Final dense layer
    x = Dense(units=len(classes), kernel_initializer="glorot_uniform",
              kernel_regularizer=l2(1e-08),
             bias_initializer="glorot_uniform")(x)
    
    prediction = Activation(activation="softmax")(x)
    
    model = Model(inputs = base_model.input, outputs=prediction)
    
    ### added
    for layer in base_model.layers:
        layer.trainable = False
    ### added    
    return model

In [ ]:
import scipy.io as sio

def load_data(path, data_type, data_class):
    raw = sio.loadmat(path)
    if data_class == "train":
        if data_type == "data":
            cars_data = [(row[5][0][:],row[4][0][0],row[0][0][0],row[1][0][0],row[2][0][0],row[3][0][0]) for row in raw['annotations'][0]]
            return cars_data

        if data_type == "class":
            cars_classes = [(row[0]) for row in raw['class_names'][0]]
            return cars_classes
        
    if data_class == "test":
        if data_type == "data":
            cars_data = [(row[4][0][:],"empty",row[0][0][0],row[1][0][0],row[2][0][0],row[3][0][0]) for row in raw['annotations'][0]]
            return cars_data

    # if data_class == "all":
    # 	if data_type == "class":
    # 		cars_classes = [(row[0]) for row in raw['class_names'][0]]
    # 		return cars_classes

In [ ]:
classes = load_data("/content/drive/My Drive/Colab Notebooks/Cars/devkit/cars_meta.mat", "class", "train")

In [ ]:
from keras.layers import Lambda
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, MaxPool2D, Reshape, Dense, Activation

from keras.regularizers import l2
from keras.models import Model, model_from_json, load_model, Sequential

# hyperparameters for initial training
lr = 0.1
epochs = 500
dropout_rate  = 0.5
batch_size = 64
model_name = "VGG19"
paitence = 10
optimizer_name = "sgd"
decay = 1e-8

# load model
tensArry=[]
for i in range(512):
    tensArry.append(1)
    
model = VGG16_outerproduct(filter_zero=0
                    , value_filter=1, classes=classes,dropoutrate=dropout_rate,model_name=model_name)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
# now all layers are trainable
for layer in model.layers:
    layer.trainable = True

# change LR
opt_sgd = tensorflow.keras.optimizers.SGD(
                lr=1e-3
                , decay=1e-9
                , momentum=0.9
                , nesterov=False
            )
model.compile(
    loss="categorical_crossentropy"
    , optimizer=opt_sgd
    , metrics=["categorical_accuracy"]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/Cars/final_all_transfer_learning_VGG16.h5")

In [ ]:
layer = model.layers[18]
layer.name

'lambda_New'

In [ ]:
# import os

# dogs_labels = set()

# path= "/content/drive/My Drive/Colab Notebooks/Data_Stanford_Dogs/images/Images"
# for d in os.listdir(path):
#     dogs_labels.add(d)

# len(dogs_labels)


In [ ]:
# dogs_labels = list(classes)
# # dogs_labels.sort()
# classes

# dogs_labels = list(dogs_labels)
# dogs_labels_path = [path + s for s in dogs_labels]
# # dogs_labels
classes = list(classes)
classes.sort()
classes

['AM General Hummer SUV 2000',
 'Acura Integra Type R 2001',
 'Acura RL Sedan 2012',
 'Acura TL Sedan 2012',
 'Acura TL Type-S 2008',
 'Acura TSX Sedan 2012',
 'Acura ZDX Hatchback 2012',
 'Aston Martin V8 Vantage Convertible 2012',
 'Aston Martin V8 Vantage Coupe 2012',
 'Aston Martin Virage Convertible 2012',
 'Aston Martin Virage Coupe 2012',
 'Audi 100 Sedan 1994',
 'Audi 100 Wagon 1994',
 'Audi A5 Coupe 2012',
 'Audi R8 Coupe 2012',
 'Audi RS 4 Convertible 2008',
 'Audi S4 Sedan 2007',
 'Audi S4 Sedan 2012',
 'Audi S5 Convertible 2012',
 'Audi S5 Coupe 2012',
 'Audi S6 Sedan 2011',
 'Audi TT Hatchback 2011',
 'Audi TT RS Coupe 2012',
 'Audi TTS Coupe 2012',
 'Audi V8 Sedan 1994',
 'BMW 1 Series Convertible 2012',
 'BMW 1 Series Coupe 2012',
 'BMW 3 Series Sedan 2012',
 'BMW 3 Series Wagon 2012',
 'BMW 6 Series Convertible 2007',
 'BMW ActiveHybrid 5 Sedan 2012',
 'BMW M3 Coupe 2012',
 'BMW M5 Sedan 2010',
 'BMW M6 Convertible 2010',
 'BMW X3 SUV 2012',
 'BMW X5 SUV 2007',
 'BMW X6

In [ ]:
# dogs_labels = list(dogs_labels)
# dogs_labels.sort()
# dogs_labels

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

def output(location, new_model):
    img = load_and_preprocess_image(location)
    img = np.expand_dims(img, axis=0)
    answer = new_model.predict(img)
    y_class = answer.argmax(axis = -1)
    
    top_3 = np.argsort(answer[0])[:-4:-1]
    # for i in range(3):
    #     print(" ({:.3})".format(answer[0][top_3[i]]))
    
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = classes[y]
    # print(res)
#     print(" ({:.3})".format(answer[0][top_3[0]]))
    pred_prob = answer[0][top_3[0]]
    return float(pred_prob), res

In [ ]:
location = "/content/drive/My Drive/Colab Notebooks/Cars/cars_classes_top_cropped/Audi TT Hatchback 2011/00767.jpg"
output(location, model)

In [ ]:
# %cd /content
# !wget http://ai.stanford.edu/~jkrause/car196/cars_train.tgz
# !wget http://ai.stanford.edu/~jkrause/car196/cars_test.tgz
# !wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
# !tar zxf car_devkit.tgz
# !tar zxf cars_train.tgz
# !tar zxf cars_test.tgz
# !wget http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat -O devkit/cars_test_annos_withlabels.mat

/content
--2021-12-31 10:11:31--  http://ai.stanford.edu/~jkrause/car196/cars_train.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 979269282 (934M) [application/x-gzip]
Saving to: ‘cars_train.tgz’

cars_train.tgz      100%[===================>] 933.90M  20.1MB/s    in 38s     

2021-12-31 10:12:08 (24.9 MB/s) - ‘cars_train.tgz’ saved [979269282/979269282]

--2021-12-31 10:12:08--  http://ai.stanford.edu/~jkrause/car196/cars_test.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977350468 (932M) [application/x-gzip]
Saving to: ‘cars_test.tgz’

cars_test.tgz       100%[===================>] 932.07M  18.7MB/s    in 68s     

2021-12-31 10:13:17 (13.7 MB/s) - ‘cars_test.tgz’ saved [977350468/9773

# Identifying top prototypes cars

In [ ]:
# import cv2
# import os
# from shutil import copyfile
# from pathlib import Path

# #folder="/content/drive/My Drive/Colab/1KDataSet_Part1"
# folder = "/content/cars_train/"
# folder_Copy = "/content/drive/My Drive/Colab Notebooks/Cars/Cars_classes_top/"

# last_fileName=""

# #images = []
# imagesFileNames = []
# # for filename in os.listdir(folder):
#     # if filename not in os.listdir(folder_Copy):
# for img in os.listdir(folder):
#   pred = output(folder+"/"+img, model)
#   if pred[0] >= 0.9 :
#       print(folder+"/"+img, model)
#       print('path', folder+"/"+img)
#       print('prob=', output(folder+"/"+img, model)[0])
#       src = folder+"/"+img
#       dst = folder_Copy+ pred[1]+"/"+img
#       # Path(folder_Copy+filename+"/").mkdir(parents=True, exist_ok=True)
#       # copyfile(src, dst)
#       print(pred[1])
# #             print(img)
#   if len(folder_Copy)==196:
#     break;
#   #     img = cv2.imread(os.path.join(folder,filename))
#   #     if img is not None:
#   #         #images.append(img)
#     imagesFileNames.append(pref[1])
# imagesFileNames.sort()        
# # print(imagesFileNames[0])
# #images = load_images_from_folder(folder)


In [ ]:
import cv2
import os
from shutil import copyfile
from pathlib import Path


folder = "/content/drive/My Drive/Colab Notebooks/Cars/cars_classes_top_cropped/"

last_fileName=""
#images = []
imagesFileNames = []
for filename in os.listdir(folder):
    # for img in os.listdir(folder+filename):
    imagesFileNames.append(filename)
    print(filename)
imagesFileNames.sort()
print(len(imagesFileNames))

In [ ]:
arryWeights_last_After =[]
for i in range(512):
  arryWeights_last_After.append(i)
for i in range(512):
  arryWeights_last_After[i]=1

def custom_layer_last(tensor):
    return tensor * arryWeights_last_After[i]
# vgg_weights = VGG16(    input_shape = IMAGE_SIZE + [3], weights = 'imagenet',
#     include_top = False).get_weights()

In [ ]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    model = Model(inputs=layers[0].input, outputs=x) ### inputs instead of input in this version
    return model

# This part for computing the CI and storing it into Dictionnary in tx file

In [ ]:
from keras.layers import Lambda
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, MaxPool2D 

# from kerassurgeon import Surgeon
# from kerassurgeon import identify
# from kerassurgeon import utils
# from kerassurgeon.operations import delete_channels
# dctOfAllDictionClasses = {}

weights_dic = {}
NO_CLASS = 196
imgCount = 0
indexImg=0
# img_path = 'Mountain_Bike/[image-net.org][27]443691662_09f12b8e37.jpg'
for imgFileName in imagesFileNames:
    dctOfAllDictionClasses = {}
    imgPath = folder + imagesFileNames[indexImg] + "/" + os.listdir(folder + imagesFileNames[indexImg])[0]
    print(imgPath)
    res = output(imgPath, model)
    # print('res[0]', res[0])
    # print('res[1]', res[1])
    originalName = imgFileName
    # originalName = res[1]
    # if '.jpg' in originalName:
    #     originalName = originalName.replace('.jpg','')
    print('orign name:', originalName)
    class_Id = res[1]
    orig_acc = res[0]
    print('origin acc', orig_acc)
    from keras.layers import Lambda
    weights_dic = {}
    K.set_learning_phase(0)
    
    import time
    start_time = time.time()
    for i in range(512):
        print ('filt{}'.format (i))
        # arryWeights_last_After[i]=0
        # def custom_layer_last(tensor):
        #   return tensor * arryWeights_last_After[i]
        filt = i

        new_model = VGG16_outerproduct(filter_zero=i
                    , value_filter=0, classes=classes,dropoutrate=dropout_rate,model_name=model_name)

        # now all layers are trainable
        for layer in new_model.layers:
            layer.trainable = True

        # change LR
        opt_sgd = tensorflow.keras.optimizers.SGD(
                        lr=1e-3
                        , decay=1e-9
                        , momentum=0.9
                        , nesterov=False
                    )
        new_model.compile(
            loss="categorical_crossentropy"
            , optimizer=opt_sgd
            , metrics=["categorical_accuracy"]
        )

        new_model.load_weights('/content/drive/My Drive/Colab Notebooks/Cars/final_all_transfer_learning_VGG16.h5')

        new_res = output(imgPath, new_model)
        # print('res[0]', new_res[0])
        # print('res[1]', new_res[1])
    
        New_accu = new_res[0]
        print(" ({:.3})".format(New_accu))
        print('label name:', new_res[1])
        print ('acc after purning (%.2f%%)', New_accu)
        print('difference: ', (orig_acc - New_accu))
        #weights_dic[filt][0] = 1/(k+1)  # resp
        tuple = orig_acc - New_accu, 1
        #weights_dic[filt] = original_loss[1] - loss[1]  # prob
        weights_dic[filt] = tuple
        # arryWeights_last_After[i]=1
        K.clear_session ()

    weights_dic_sort = sorted (weights_dic.items (), key=lambda kv: kv[1], reverse=True)
    #     print ('Resp and loss for conv layer {}\n'.format (1), weights_dic_sort)
    # arryOfDictFiltByLayer.append(weights_dic_sort)
    #K.backend.clear_session ()
    print("--- %s seconds ---" % (time.time() - start_time))
    pairKeyValue = { originalName: weights_dic_sort}
    dctOfAllDictionClasses = {}
    dctOfAllDictionClasses.update(pairKeyValue)
    print('pair ley:', pairKeyValue)
    import json
    #     with open('FIltersRespAll', 'w') as fout:
    #         json.dump(str(lstOfAllDictionClasses), fout)
    # with open('/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/Filt_Resp_Birds_BCNN.txt', 'a') as fout:
    with open('/content/drive/My Drive/Colab Notebooks/Cars/Filt_Resp_Cars_Crop.txt', 'a') as fout:

        fout.write("\n" + str(dctOfAllDictionClasses) + "\n") 
        print('Last Image ID:', imagesFileNames[imgCount])
    imgCount+=1
    indexImg+=1
    


/content/drive/My Drive/Colab Notebooks/Cars/cars_classes_top_cropped/Bentley Arnage Sedan 2009/01076.jpg
orign name: Bentley Arnage Sedan 2009
origin acc 0.9610258340835571
filt0


/usr/local/lib/python3.7/dist-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
difference:  -0.0001246333122253418
filt26
 (0.902)
label name: Bentley Mulsanne Sedan 2011
acc after purning (%.2f%%) 0.9024918675422668
difference:  0.0007460117340087891
filt27
 (0.902)
label name: Bentley Mulsanne Sedan 2011
acc after purning (%.2f%%) 0.902341902256012
difference:  0.0008959770202636719
filt28
 (0.903)
label name: Bentley Mulsanne Sedan 2011
acc after purning (%.2f%%) 0.9032378792762756
difference:  0.0
filt29
 (0.904)
label name: Bentley Mulsanne Sedan 2011
acc after purning (%.2f%%) 0.9038646817207336
difference:  -0.0006268024444580078
filt30
 (0.903)
label name: Bentley Mulsanne Sedan 2011
acc after purning (%.2f%%) 0.9032378792762756
difference:  0.0
filt31
 (0.903)
label name: Bentley Mulsanne Sedan 2011
acc after purning (%.2f%%) 0.9032378792762756
difference:  0.0
filt32
 (0.899)
label name: Bentley Mulsanne Sedan 2011
acc after purning (%.2f%%) 0.8987672328948975
difference:  0.004470646381378174
filt33
 (